# Train a Classifier to detect traffic lights

##### Udacity Self-Driving Car Nanodegree
##### project system integration
##### team vulture

This approach follows very closely the one documented in https://github.com/cena0805/ros-traffic-light-classifier as this is almost what we need.

Rainer Bareiss, V0.5, August 27th, 2017, Stuttgart

## 1 imports

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import RMSprop


from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

#from visual_callbacks import AccLossPlotter
import numpy as np
import glob
import os
from PIL import Image
import collections
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras
from keras.models import Model
from keras.layers import Input, Activation, merge
from keras.layers import Flatten, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import AveragePooling2D
from keras.optimizers import Adam
import keras.backend as K
K.set_image_dim_ordering('tf')

# import neural nets
import model 
from model import *

# import parameters
from parameters import *

##########
#import keras
#from keras.models import Model
#from keras.layers import Input, Activation, merge
#from keras.layers import Flatten, Dropout
#from keras.layers import Convolution2D, MaxPooling2D
#from keras.layers import AveragePooling2D
#from keras.optimizers import Adam
#import keras.backend as K

Using TensorFlow backend.


## 2 parameters

In [2]:
print("batch_size:       ",batch_size)
print("nb_epoch:         ",nb_epoch)
print("nb_train_samples: ",nb_train_samples)
print("save_img:         ",save_img)

batch_size:        64
nb_epoch:          25
nb_train_samples:  2000
save_img:          False


## 3 load & explore model

In [3]:
# Option 1: original model from https://github.com/cena0805/ros-traffic-light-classifier
model = model.get_model()

# Option 2: squeezenet from https://github.com/mynameisguy/TrafficLightChallenge-DeepLearning-Nexar/blob/master/train.py
#model = model.SqueezeNet(4, (IMAGE_HEIGHT, IMAGE_WIDTH, 3))

# generate model summary
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 224, 224, 32)  896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 224, 224, 32)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 222, 222, 32)  9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 222, 222, 32)  0           convolution2d_2[0][0]            
___________________________________________________________________________________________

## 4 load data and split into training, validate and test data

In [4]:
# set data pathes
# input & output folder
mydir_in = '/Users/rainerbareiss/Library/Mobile Documents/M6HJR9W95L~com~textasticapp~textastic/Documents/02-Udacity/carND/03-term3/P3-CarND-Capstone/00-team_vulture/SDC-System-Integration/classifier/'
mydir_out = '/Users/rainerbareiss/Downloads/traffic_light_images_udacity/'

In [5]:
# Option: Udacity data
myfiles = glob.glob(mydir_in+"/06-test_images_udacity/*/*.*")

#print(myfiles)
train_samples, validation_samples = train_test_split(myfiles, test_size=0.2)
print(len(train_samples))
print(len(validation_samples))

1488
373


## 5 prepare data for training 

### input

/images:
    /red
    /green
    /unknown
    
### output (augmented)

/train:
    /red
    /green
    /unknown
    
/test:
    /red
    /green
    /unknown

### store training data into separate folder

In [6]:
# https://stackoverflow.com/questions/3397157/how-to-read-a-raw-image-using-pil

In [7]:
for ifile in train_samples:
    # read data
    file = Image.open(ifile)
    imgSize = file.size
    rawData = file.tobytes()
    
    # get class & file name
    classdir = ifile.split("/")[-2]   # class name
    classname = ifile.split("/")[-1]  # name of the image file
    #print(classdir + ' ---> ' + classname)
    
    # generate new file names
    ofile = mydir_out + 'train/' + classdir + '/' + classname
    #print(ofile)
    
    # save in new folder
    img = Image.frombytes('RGB', imgSize, rawData)
    if (save_img): img.save(ofile)
    #print(ifile)

In [8]:
datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.1,
                             fill_mode='nearest', rescale=1. / 255)
#train_image_data_gen = datagen.flow_from_directory(mydir_out+'train', target_size=(64, 64), classes=['green', 'red', 'unknown'], batch_size=batch_size)
#train_image_data_gen = datagen.flow_from_directory(mydir_out+'train', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), classes=['green', 'red', 'unknown'], batch_size=batch_size)
train_image_data_gen = datagen.flow_from_directory(mydir_out+'train', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), classes=['0-red','1-yellow','2-green','4-unknown'], batch_size=batch_size)

print("number of image samples: ",train_image_data_gen.nb_sample)

Found 1488 images belonging to 4 classes.
number of image samples:  1488


### store test data into separate folder

In [9]:
y_ground_truth = []

for ifile in validation_samples:
    # read data
    file = Image.open(ifile)
    imgSize = file.size
    rawData = file.tobytes()
    
    # get class & file name
    classdir = ifile.split("/")[-2]   # class name
    classname = ifile.split("/")[-1]  # name of the image file
    #print(classdir + ' ---> ' + classname)
    
    # generate new file names
    ofile = mydir_out + 'test/' + classdir + '/' + classname
    #print(ofile)
    
    # save in new folder
    img = Image.frombytes('RGB', imgSize, rawData)
    if (save_img): img.save(ofile)
    #print(ifile)
    
    # generate vector of ground_truth
    if (classdir == '0-red'): y_ground_truth.append([1,0,0,0])
    if (classdir == '1-yellow'): y_ground_truth.append([0,1,0,0])
    if (classdir == '2-green'): y_ground_truth.append([0,0,1,0])
    if (classdir == '4-unknown'): y_ground_truth.append([0,0,0,1])

In [10]:
print(len(y_ground_truth))

373


In [11]:
#test_image_data_gen = datagen.flow_from_directory(mydir_out+'test', target_size=(64, 64), classes=['green', 'red', 'unknown'],batch_size=batch_size)
#test_image_data_gen = datagen.flow_from_directory(mydir_out+'test', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), classes=['green', 'red', 'unknown'],batch_size=batch_size)
test_image_data_gen = datagen.flow_from_directory(mydir_out+'test', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), classes=['0-red','1-yellow','2-green','4-unknown'],batch_size=batch_size)



#datagen_test = ImageDataGenerator(rescale=1. / 255)
#test_image_data_gen = datagen_test.flow_from_directory(mydir_out+'test', target_size=(IMAGE_WIDTH, IMAGE_HEIGHT), classes=['0-red','1-yellow','2-green','4-unknown'],batch_size=batch_size)



print("number of image samples: ",test_image_data_gen.nb_sample)

Found 373 images belonging to 4 classes.
number of image samples:  373


## 6 train model

In [12]:
# no. lr     batch images <> model     <> Opt.    <> epochs  s/epoch <> loss           acc         >>> test acc.   
# ===================================================================================================================
# 001 ---    64    all    <> get_model <> RMSPROP <> 25      600s    <> loss: 0.25   - acc: 0.85,  >>> 44%
# 002 0.0001 64    2000   <> get_model <> Adam     <> 25       50s    <> loss: 0.3328 - acc: 0.8896 >>> 42.4%
# 003 0.001  64    2000   <> get_model <> Adam     <> 25       50s    <> loss: 0.1373 - acc: 0.9585 >>> 48.4% / 49.6%
# 004 0.001  16    2000   <> get_model <> Adam     <> 25       80s    <> loss: 0.1613 - acc: 0.9545 >>> 47.5% / 50.1%
# 005 0.001  128   2000   <> get_model <> Adam     <> 25       50s    <> loss: 0.1627 - acc: 0.9521 >>> 42.9%
# 006 0.005  64    2000   <> get_model <> Adam     <> 25       50s    <> loss: 0.1304 - acc: 0.9673 >>> 44.3%
# 007 0.001  64    all    <> get_model <> Adam     <> 50       850s   <> loss: 0.0919 - acc: 0.9737 >>> 50.0% <<<
# 008 0.001  64    2000   <> squeezze  <> Adam     <> 3        380s   <> loss: 0.7187 - acc: 0.7310 >>> 51.2% / 54.8
# 009 0.01   64    2000   <> squeezze  <> Adam     <> 3        371s   <> loss: 5.7216 - acc: 0.6450 >>> 67.0%
# 010 0.005  64    2000   <> squeezze  <> Adam     <> 3        370s   <> loss: 0.8576 - acc: 0.6704 >>> 65.8% / 65.8%
# 011 0.005  64    2000   <> squeezze  <> Adam     <> 25       349s   <> loss: 5.7216 - acc: 0.6450 >>> 66.1%
# using Udacity data from here onwards
# 012 0.001  64    all-ud <> squeezze  <> Adam     <> 25       349s   <> loss: 0.4217 - acc: 0.8324 >>> DATA MISMATCH
# 013 0.001  64    all-ud <> squeezze  <> Adam     <> 25       338s   <> loss: 0.6871 - acc: 0.7021 >>> DATA MISMATCH
# 014 0.001  64    all-ud <> squeezze  <> SGD      <> 25       260s   <> loss: 1.1941 - acc: 0.4973 >>> 51.7%
# 015 0.001  64    all-ud <> get_model <> SGD      <> 3        500s   <> loss: 1.0270 - acc: 0.6028 >>> 46.4% 
# 016 0.001  64    all-ud <> get_model <> SGD      <> 25       450s   <> loss: 0.6067 - acc: 0.7272 >>> 38.1% 

# test accuracy is determined for predicting a completly unseen test dataset splitted from the original dataset

In [13]:
# select optimizer compile model
# Option 1: rmsprop
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Option 2: Adams Optimizer
#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01) # original
###adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01) # original
#adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.01) 
#adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)

########## Option 1: Adam
##adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)  # proposed original values
##model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
##########

########## Option 2: RMSprop
##rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
##model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])
##########

########## Option 3: SGD
sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
##########

# test model with less data
#model.fit_generator(train_image_data_gen, nb_epoch=nb_epoch, samples_per_epoch=nb_train_samples)

# full model
model.fit_generator(train_image_data_gen, nb_epoch=nb_epoch, samples_per_epoch=train_image_data_gen.nb_sample)

Epoch 1/25
1488/1488 [==============================] - 479s - loss: 1.2104 - acc: 0.4906     

## 7 save model

In [14]:
#model.save('./10-model-checkpoints/008_traffic_lights_classifier_squeezze_sgd_epoch3.h5')
model_name = '016_traffic_lights_classifier.h5'
model.save(model_name)

## 8 load model 

In [15]:
# helper functions

# Min-Max scaling for grayscale image data
# http://sebastianraschka.com/Articles/2014_about_feature_scaling.html#about-min-max-scaling
def normalize_scale(X):
    a = 0
    b = 1.0
    return a + X * (b-a) / 255

# http://lamda.nju.edu.cn/weixs/project/CNNTricks/CNNTricks.html
def standardize(X):
    X -= np.mean(X, axis = 0) # zero-center
    X /= np.std(X, axis = 0) # normalize
    return (X)

# preprocessing pipeline
def prepare_data(X):
    
    # step 1: convert to gray image
    #X = rgb2gray(X)
    
    # add the dimensions again in order for working with LeNet
    X = np.reshape(X, (X.shape[0],64,64,-1))
    
    # Step 2: Normalize RGB-values into interval [-1,1]    
    X = normalize_scale(X)

    # Step 3: Convert data to values with zero mean and std = 1
    X = standardize(X)
    return X

In [16]:
def reshape_image(image):
  x = img_to_array(image.resize((IMAGE_WIDTH, IMAGE_HEIGHT), Image.ANTIALIAS))
  #x = prepare_data(x)
  return x[None, :]

def show(filename):
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg
    #image = Image.open(filename)
    #image.show()
    %pylab inline
    img=mpimg.imread(filename)
    imgplot = plt.imshow(img)
    plt.show()

In [17]:
model_predict = load_model(model_name)

## 9 analyse performance of model

In [18]:
# Option 1: orignal

In [19]:
# apply prediction over all test data 
myfiles = glob.glob(mydir_out+"/test/*/*.*")
my_pred = []

for ifile in myfiles:
    my_img = load_img(ifile)
    my_prob = model_predict.predict(reshape_image(my_img))
    my_pred.append([my_prob[0][0],my_prob[0][1],my_prob[0][2],my_prob[0][3]])
    #print(max(my_prob))
print(len(my_pred))

373


In [20]:
# generate array of indices showing the class number of prediction
my_pred_ind = []

for ipred in my_pred:
        imax = np.argwhere(ipred == np.amax(ipred))
        ##print(ipred, "     ", imax[0][0])
        my_pred_ind.append(imax[0][0])
        #if (imax[0][0] != 2): my_pred_ind.append(imax[0][0])
        
print(len(my_pred_ind))

373


In [21]:
#print(y_ground_truth)

In [22]:
# generate array of indices showing the ground truth class
my_true_ind = []

for itrue in y_ground_truth:
        imax = np.argwhere(itrue == np.amax(itrue))
        ##print(ipred, "     ", imax[0][0])
        my_true_ind.append(imax[0][0])
        #if (imax[0][0] != 2): my_true_ind.append(imax[0][0])
        
print(len(my_true_ind))

373


In [23]:
# check elementwise for equality and store result in vector
comp = np.equal(my_pred_ind, my_true_ind)
#print("array of comparision: ",comp)

# count occurences of true
collections.Counter(comp)
mytrues = Counter(comp)
if (len(comp) != 0): 
    print("Accuracy of predicting unseen traffic lights = ", mytrues[1]/len(comp)*100,"%")
else:
    print("no red or green lights detected")

Accuracy of predicting unseen traffic lights =  40.21447721179624 %


In [24]:
# Option 2: following https://stackoverflow.com/questions/44156974/how-to-interpret-keras-predict-generator-output

In [25]:
# set directory for test data
validation_data_dir=mydir_out+'test'

# setup data generator
test_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=batch_size,
    classes=['0-red','1-yellow','2-green','4-unknown'])
    #class_mode='binary')
print(len(validation_generator.filenames))

# generate predictions
predictions=model_predict.predict_generator(validation_generator,len(validation_generator.filenames));
#print(predictions)

Found 373 images belonging to 4 classes.
373


In [26]:
# apply prediction over all test data 
myfiles = glob.glob(mydir_out+"/test/*/*.*")
my_pred = []

for ipred in predictions:
    #my_img = load_img(ifile)
    #my_prob = model_predict.predict(reshape_image(my_img))
    my_pred.append(ipred)
    #print(max(my_prob))
print(len(my_pred))

373


In [27]:
# generate array of indices showing the class number of prediction
my_pred_ind = []

for ipred in my_pred:
        imax = np.argwhere(ipred == np.amax(ipred))
        ##print(ipred, "     ", imax[0][0])
        my_pred_ind.append(imax[0][0])
        #if (imax[0][0] != 2): my_pred_ind.append(imax[0][0])
        
print(len(my_pred_ind))

373


In [28]:
#print(predictions)
# check elementwise for equality and store result in vector
comp = np.equal(my_pred_ind, my_true_ind)
#print("array of comparision: ",comp)

# count occurences of true
collections.Counter(comp)
mytrues = Counter(comp)
if (len(comp) != 0): 
    print("Accuracy of predicting new traffic lights from simulator = ", mytrues[1]/len(comp)*100,"%")
else:
    print("no red or green lights detected")

Accuracy of predicting new traffic lights from simulator =  38.069705093833775 %
